# Evaluation


# Produce results for a sequence
Saves inference for test images in evaluation folder

In [41]:
import os
import time
import numpy as np
from pg_networks.gcn import GCN
from pg_datasets.davis_2016 import DAVIS2016
import torch
from torch.utils.data.sampler import SequentialSampler
from torch_geometric.data import DataLoader
import cv2
from Combination_OSVOS_contour.final_prediction import compute_combo_img, plot_all
from evaluation.metrics import db_eval_iou, db_eval_boundary, db_eval_t_stab

In [42]:
#Adapt first 3 lines!
seq_name = ['bear']
additional_comment = 'overfit_model'
model_path = 'pg_models/Bear_overfit_model.pth'

original_images = os.path.join('DAVIS_2016/DAVIS/JPEGImages/480p/', seq_name[0])
OSVOS_images = os.path.join('OSVOS_PyTorch/models/Results', seq_name[0])
annotations = os.path.join('DAVIS_2016/DAVIS/Annotations/480p/', seq_name[0])
save_folder = os.path.join('evaluation', seq_name[0] + '_' + additional_comment)

if not os.path.exists(save_folder):
    os.mkdir(save_folder)

# Dataset parameters
PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH = 'pg_datasets/DAVIS_2016'
CONTOURS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Contours/480p'
IMAGES_FOLDERS_PATH = 'DAVIS_2016/DAVIS/JPEGImages/480p'
TRANSLATIONS_FOLDERS_PATH = 'DAVIS_2016/DAVIS/Translations/480p'
EPOCHS_WO_AVEGRAD = 200
LAYER = 9
K = 32
NUM_SEQUENCES = 5

eval_set = DAVIS2016(PYTORCH_GEOMETRIC_DAVIS_2016_DATASET_PATH, 
                CONTOURS_FOLDERS_PATH, IMAGES_FOLDERS_PATH, TRANSLATIONS_FOLDERS_PATH, 
                LAYER, K, EPOCHS_WO_AVEGRAD, [], [], seq_name, train=False)

model = GCN(in_channels=eval_set[0].num_features, out_channels=eval_set[0].y.shape[1])
model.load_state_dict(torch.load(model_path))
model.eval()
model.double()

GCN(
  (conv1): GCNConv(256, 512)
  (conv2): GCNConv(512, 512)
  (conv3): GCNConv(512, 1024)
  (lin1): Linear(in_features=1024, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=256, bias=True)
  (lin3): Linear(in_features=256, out_features=2, bias=True)
)

In [44]:
#Make predictions for each step
eval_loader = DataLoader(eval_set, batch_size=1, shuffle=False, sampler=SequentialSampler(eval_set))
for sample_number, data in enumerate(eval_loader):
       
    with torch.no_grad():
        translation_pred = model(data)
        
    if sample_number <= 9:
        filename = '0000' + str(sample_number) + '.png'
        annotation_filename = '0000' + str(sample_number) + '.png'
    elif sample_number <= 99:
        filename = '000'  + str(sample_number) + '.png'
    elif sample_number <= 999:
        filename = '00'   + str(sample_number) + '.png'
    
    print(filename)

    OSVOS_img = cv2.imread(os.path.join(OSVOS_images, filename))
    annotation_img = cv2.imread(os.path.join(annotations, filename), cv2.IMREAD_GRAYSCALE)
    contour = data.contour
    predicted_contour = contour.type(torch.DoubleTensor).add(translation_pred)
    final_prediction = compute_combo_img(predicted_contour, OSVOS_img)
    cv2.imwrite(os.path.join(save_folder, filename), final_prediction*255)
    
    #Compute IOU 
    iou = db_eval_iou(annotation_img, final_prediction)
    print('IOU:', iou)    
    #Compute contour accuracy
    contour_accuracy = db_eval_boundary(final_prediction, annotation_img)
    print('Contour Accuracy:', contour_accuracy)
    #Compute temporal stability
#     if sample_number > 0:
#         temporal_stability = db_eval_t_stab(final_prediction_last_timestep, final_prediction_last_timestep, timing=True)
#         print('Temporal Stability:', temporal_stability)
#     final_prediction_last_timestep = final_prediction

00000.png
IOU: 0.9386774661028726
Contour Accuracy: 0.9970162956162497
00001.png
IOU: 0.9340652747780177
Contour Accuracy: 0.9997556804300024
00002.png
IOU: 0.9377699373441207
Contour Accuracy: 0.9955156950672646
00003.png
IOU: 0.9405383277044315
Contour Accuracy: 1.0
00004.png
IOU: 0.9463160948118681
Contour Accuracy: 0.9846069268829027
00005.png
IOU: 0.94509182631173
Contour Accuracy: 0.9937782805429864
00006.png
IOU: 0.9452606285437262
Contour Accuracy: 0.977112676056338
00007.png
IOU: 0.9462493621364177
Contour Accuracy: 0.9819277108433735
00008.png
IOU: 0.9452185937666205
Contour Accuracy: 0.9842105263157895
00009.png
IOU: 0.9456378966992924
Contour Accuracy: 0.987467210725736
00010.png
IOU: 0.9461562663869952
Contour Accuracy: 0.9835320840431573
00011.png
IOU: 0.9462068679151269
Contour Accuracy: 0.9797200799771494
00012.png
IOU: 0.9456241889971371
Contour Accuracy: 0.9648382559774965
00013.png
IOU: 0.9412088921550615
Contour Accuracy: 0.9677760169043846
00014.png
IOU: 0.94308458

KeyboardInterrupt: 